In [4]:
# Importation des Bibliotheques
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [6]:
# Chargement des Donnees
intents_file = open('Liste_Intentions.json', encoding='utf-8').read()
intents = json.loads(intents_file)
listInt = intents['intentions']
for i in  range(len(listInt)):
    print(i,listInt[i]['tag'] )
    listInt[i]['questions']

0 salutations
1 piecesAfournir
2 domaine 
3 specialisation
4 diplomesDelivres
5 modalite
6 biblioteque
7 localisation
8 conditionsDAdmissibilites
9 debutDesCours
10 remerciements
11 auRevoir
12 


In [19]:
# Pretraitement
nltk.download('wordnet')
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

for intent in intents['intentions']:
    for pattern in intent['questions']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        documents.append((word, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

words[:5]

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('checkpoints/words.pkl','wb'))
pickle.dump(classes,open('checkpoints/classes.pkl','wb'))



[(['Bonjour'], 'salutations'), (['Comment', 'allez-vous', '?'], 'salutations'), (['Salut'], 'salutations'), (['Quoi', 'de', 'neuf'], 'salutations'), (['Salut', '!'], 'salutations'), (['Ça', 'va', '?'], 'salutations'), (['quelle', 'sont', 'les', 'différentes', 'pieces', 'demander', '?'], 'piecesAfournir'), (['quelle', 'sont', 'les', 'papiers', 'que', 'vous', 'demandez', '?'], 'piecesAfournir'), (['dites', 'moi', 'les', 'pieces', 'à', 'amener', 'pour', "s'inscrir", '?'], 'piecesAfournir'), (['donner', 'moi', 'les', 'papiers', 'demander', 'lors', 'de', "l'inscription", '?'], 'piecesAfournir'), (['citez', 'moi', 'les', 'pieces', 'dont', 'on', 'a', 'besoin', 'pour', "s'inscrir", '?'], 'piecesAfournir'), (['pouriez', 'vous', 'me', 'donner', 'des', 'renseignement', 'sur', 'les', 'papiers', 'à', 'fournir', '?'], 'piecesAfournir'), (['votre', 'formation', 'est', 'accée', 'sur', 'quel', 'domaine', '?'], 'domaine '), (['quels', 'sont', 'les', 'domaines', 'sur', 'lesquels', 'vous', 'dispensez', 'v

[nltk_data] Downloading package wordnet to C:\Users\The Tech
[nltk_data]     Alchemist\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
# Creation des donnees d'entrainement
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
train_x, train_y = zip(*training)
train_x, train_y = np.array(train_x), np.array(train_y)
print("Training data is created")

Training data is created


In [25]:
# Entrainement du model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=900, batch_size=8)
model.save('checkpoints/chatbot_model.h5', hist)
print("model is created")

Epoch 1/900
7/7 [==============================] - 3s 12ms/step - loss: 2.5375 - accuracy: 0.0727
Epoch 2/900
7/7 [==============================] - 0s 5ms/step - loss: 2.5249 - accuracy: 0.0727
Epoch 3/900
7/7 [==============================] - 0s 11ms/step - loss: 2.4061 - accuracy: 0.1818
Epoch 4/900
7/7 [==============================] - 0s 13ms/step - loss: 2.3636 - accuracy: 0.1636
Epoch 5/900
7/7 [==============================] - 0s 10ms/step - loss: 2.2505 - accuracy: 0.2364
Epoch 6/900
7/7 [==============================] - 0s 13ms/step - loss: 2.2188 - accuracy: 0.2364
Epoch 7/900
7/7 [==============================] - 0s 9ms/step - loss: 2.0606 - accuracy: 0.3455
Epoch 8/900
7/7 [==============================] - 0s 14ms/step - loss: 1.9222 - accuracy: 0.4545
Epoch 9/900
7/7 [==============================] - 0s 17ms/step - loss: 2.0553 - accuracy: 0.2909
Epoch 10/900
7/7 [==============================] - 0s 13ms/step - loss: 1.8456 - accuracy: 0.4364
Epoch 11/900
7/7 [===

C:\Users\The Tech Alchemist\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


model is created


In [26]:
import nltk
import pickle
import numpy as np
from nltk.stem import WordNetLemmatizer
from keras.models import load_model
import json
import random


lemmatizer = WordNetLemmatizer()
model = load_model('checkpoints/chatbot_model.h5')

intents = json.loads(open('Liste_Intentions.json', encoding='utf-8').read())
words = pickle.load(open('checkpoints/words.pkl','rb'))
classes = pickle.load(open('checkpoints/classes.pkl','rb'))

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

    
def bag_of_words(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                 bag[i] = 1
                 if show_details:
                     print ("found in bag: %s" % word)
    return(np.array(bag))


def predict_class(sentence):
     p = bag_of_words(sentence, words,show_details=False)
     res = model.predict(np.array([p]))[0]
     ERROR_THRESHOLD = 0.25
     results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

     results.sort(key=lambda x: x[1], reverse=True)
     return_list = []
     for r in results:
         return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
     return return_list

def getResponse(ints, intents_json):
     tag = ints[0]['intent']
     list_of_intents = intents_json['intentions']
     for i in list_of_intents:
         if(i['tag']== tag):
             result = random.choice(i['réponses'])
             break
     return result

In [27]:


#Creating tkinter GUI
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 
        ints = predict_class(msg)
        res = getResponse(ints, intents)
        ChatBox.insert(END, "Bot: " + res + '\n\n')           
        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)

root = Tk()
root.title("Chatbot")
root.geometry("400x500")
root.resizable(width=False, height=False)

#Create Chat window
ChatBox = Text(root, bd=0, bg="white", height="20", width="50", font="Arial",)
ChatBox.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), 
                    text="Send", width="10", height="4",
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",
                    fg='#000000',command= send )


#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="40", height="4", font="Arial")

#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
root.mainloop()


1/1 [==============================] - 0s 49ms/step
